<a href="https://colab.research.google.com/github/daigo38/create-pinecone-index/blob/main/create_pinecone_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
packages = """
langchain
openai
tiktoken
urllib3<2
pinecone-client

unstructured
selenium
pypdf
pypdfium2
pdf2image
tabulate
"""

with open(f"./requirements.txt", "w") as f:
    f.write(packages)

!pip install -r requirements.txt
!mkdir load_files

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

In [ ]:
import os
from logging import getLogger
logger = getLogger(__name__)
import pinecone

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import (
    SeleniumURLLoader,
    PyPDFLoader,
    UnstructuredPDFLoader,
    CSVLoader,
    UnstructuredFileLoader,

)
from langchain.text_splitter import (
    MarkdownTextSplitter, 
    CharacterTextSplitter
)


extension_dict = {
    '.txt': 'text',
    '.csv': 'csv',
    '.json': 'json',
    '.pdf': 'pdf',
    '.md': 'markdown',
}
special_dirs_dict = {
    "NotionDB": "notion",
}

load_dir = "/content/load_files"


class CreateIndex:
    def __init__(self):
        self.embeddings = OpenAIEmbeddings()
        self.index_name = os.environ["INDEX_NAME"]
        self.docs = []

        # initialize pinecone
        pinecone.init(
            api_key=os.environ["PINECONE_API_KEY"],  # find at app.pinecone.io
            environment=os.environ["PINECONE_ENV"]  # next to api key in console
        )
    
    def _load_pdf(self, path):
        try:
            print("try PyPDFLoader")
            loader = PyPDFLoader(path)
            documents = loader.load()
        except Exception as e:
            logger.error(f"Exception occurred: {e}", exc_info=True)
            print("try UnstructuredPDFLoader")
            loader = UnstructuredPDFLoader(path)
            documents = loader.load()
        
        text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=0)
        split_docs = text_splitter.split_documents(documents)
        return split_docs
    
    def _load_url(self, urls):
        loader = SeleniumURLLoader(urls)
        documents = loader.load()
        text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=0)
        split_docs = text_splitter.split_documents(documents)
        return split_docs
      
    def _load_csv(self, path):
        print("try CSVLoader")
        loader = CSVLoader(path)
        documents = loader.load()
        return documents
    
    def _load_text(self, path):
        print("try UnstructuredFileLoader")
        loader = UnstructuredFileLoader(path)
        documents = loader.load()
        text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=0)
        split_docs = text_splitter.split_documents(documents)
        return split_docs
    
    def _entry_documents(self, type, name):
        self.docs.append(
            {"type": type, "name": name}
        )
    
    def load_documents(self, urls=None):
        # ウェブサイトを学習
        if urls:
          self._entry_documents("url", urls)

        for root, _, files in os.walk(load_dir):
            dir_name = os.path.basename(root)
            if dir_name in special_dirs_dict:
                self._entry_documents(special_dirs_dict[dir_name], root)
                continue
            for file in files:
                file_path = os.path.join(root, file)
                _, ext = os.path.splitext(file_path)
                file_type = extension_dict.get(ext, ext)

                print(f"File name: {file_path}, File type: {file_type}")
                self._entry_documents(file_type, file_path)
    
    def create_vectordb(self):
        documents = []
        source_list = []
        for doc in self.docs:
            try:
                document = None
                if doc["type"] == "pdf":
                    document = self._load_pdf(doc["name"])
                elif doc["type"] == "markdown":
                    document = self._load_pdf(doc["name"])
                elif doc["type"] == "csv":
                    document = self._load_csv(doc["name"])
                elif doc["type"] == "url":
                    document = self._load_url(doc["name"])
                if doc["type"] == "text":
                    document = self._load_text(doc["name"])
                elif doc["type"] == "notion":
                    document = self._load_pdf(doc["name"])
                
                if document:
                    documents.extend(document)
                    source_list.append(doc["name"])
            except Exception as e:
                print(f"Error: {e}")
                return doc["name"]
        
        if documents:
            # fix metadata
            for document in documents:
                source = document.metadata.get("source")
                if source:
                    document.metadata["source"] = source.replace(load_dir+"/", "")
            # vectordb = Pinecone.from_documents(documents, self.embeddings, index_name=self.index_name)
            print(documents)
        
        return source_list

In [ ]:
os.environ["INDEX_NAME"] = "INDEX_NAME"
os.environ["PINECONE_API_KEY"] = "PINECONE_API_KEY"
os.environ["PINECONE_ENV"] = "PINECONE_ENV"
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

In [ ]:
urls = [
]
create_index = CreateIndex()

create_index.load_documents(urls=urls)
sources = create_index.create_vectordb()